In [2]:
%load_ext autoreload
%autoreload 2
from data_translator import CMAToScheduleTranslator, TestTranslator, EncodeForCMA, EncodeForGA, GAToScheduleTranslator
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment, Schedule
from solver import CMAWeightSolver, HybridGreedyAgentSolver
from visualize import visualize_schedule_demo
import objective_function

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
n_workstations, recipes, operation_times = get_data(3)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

In [4]:
earliest_slot = 0
last_slot = 100
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, last_slot, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1

In [5]:
encoder : EncodeForCMA = EncodeForCMA()
values, durations, jobs, due_dates = encoder.translate(env, orders)

In [6]:
solver : CMAWeightSolver = CMAWeightSolver(values, durations, jobs, env, orders)
solver.initialize() # doesn't do anything right now
solver.run()

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 40 (seed=104103, Wed Jan 18 12:25:22 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.000000000000000e+00 1.0e+00 4.68e-01  5e-01  5e-01 0:04.4
    2     30 2.000000000000000e+00 1.1e+00 4.57e-01  5e-01  5e-01 0:08.5
    3     45 2.000000000000000e+00 1.1e+00 4.48e-01  4e-01  5e-01 0:12.2
    4     60 2.000000000000000e+00 1.1e+00 4.38e-01  4e-01  4e-01 0:15.8
termination on tolflatfitness=1 (Wed Jan 18 12:25:38 2023)
final/bestever f-value = 2.000000e+00 2.000000e+00
incumbent solution: [3.43640857e-01 1.19778793e-01 5.09596375e-01 1.89619942e-02
 2.89750855e-01 3.37050514e-01 1.07832414e-06 1.41119278e-01 ...]
std deviations: [0.43799685 0.43689795 0.43674522 0.43822717 0.43723104 0.43818391
 0.43415299 0.43948801 ...]
Done


In [7]:
result = solver.best_solution[0]
print("Parameters of the best solution :\n {solution}".format(solution=result))
print(f'Fitness value of the solution: {solver.best_solution[1].result.fbest}')

Parameters of the best solution :
 [3.45044793e-01 6.09784950e-01 5.73876614e-03 4.07777607e-01
 4.65790876e-01 3.78577202e-01 7.87677678e-02 4.62380391e-01
 3.12642950e-01 3.35380578e-01 9.76646296e-02 2.12235992e-01
 1.01684250e-01 1.99052594e-01 5.10412576e-01 2.69570010e-01
 1.89218539e-01 1.84749508e-01 4.47249311e-01 3.48550340e-01
 2.31361837e-01 1.16815616e-02 1.11486916e-01 6.43899595e-01
 4.60996898e-01 2.19408785e-01 8.55604488e-01 8.53063021e-02
 1.76293288e-01 7.82763577e-01 2.26073085e-01 5.73711157e-02
 1.16631593e-01 7.90441696e-01 3.08949192e-05 1.93161639e-03
 3.50670058e-01 3.82746293e-01 6.80824807e-01 2.48704198e-01]
Fitness value of the solution: 2


In [8]:
schedule = CMAToScheduleTranslator().translate(result, jobs, due_dates, durations, env, orders)
schedule.created_in = env
schedule.created_for = orders
schedule.created_by = solver

In [9]:
#TODO: current solution could be a result of either a) bug in schedule building (more likely) or b) lack of feasibility checks, since they shouldn't be necessary
visualize_schedule_demo(schedule, env, orders)

In [10]:
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

AttributeError: 'NoneType' object has no attribute 'tasks'

In [11]:
values = [9,2,3,4,5,1,1,2,4,5,6,7,6,2,2,3,4,5,5,6]
sorted = []
for i in range(len(values)):
    idx = values.index(min(values)) # workstation id
    sorted.append(values[idx])
    values[idx] = float('inf')
print(sorted)
print(values)

[1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 9]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [26]:
encoder = EncodeForGA()
values, durations, all_jobs = encoder.translate(env, orders)

     
crossover = 'two_points' #NOTE: available in PyGAD: 'two_points', 'single_point', 'uniform', 'scattered'
selection = 'rws' #NOTE: available in PyGAD: 'sss' (steady state selection', 'rws' (roulette wheel), 'sus' (stochastic universal selection), 'rank' (rank selection), 'random' (random selection), 'tournament' (tournament selection)
mutation = 'swap' #NOTE: available options: 'swap', 'inversion', 'scramble', 'adaptive
#objective = 'makespan' #NOTE: available options: 'makespan', 'idle_time'
population_size = 100
max_generations = 2000

solver = HybridGreedyAgentSolver(values, durations, all_jobs, env, orders)
solver.initialize(mutation, crossover, selection, max_generations, population_size)
solver.run()

Done
Done


In [27]:
result = solver.get_best()
fitness = solver.get_best_fitness()

print("Parameters of the best solution : {solution}".format(solution=result))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(fitness)))

Parameters of the best solution : [9, 0, 9, 1, 9, 5, 9, 10, 5, 0, 5, 5, 5, 15, 5, 20, 0, 0, 9, 13, 9, 18, 9, 22, 5, 21, 5, 23, 5, 25, 5, 30, 4, 0, 4, 8, 4, 11, 4, 12, 1, 0, 1, 4, 4, 17, 9, 24, 6, 0, 1, 8, 4, 18, 9, 27, 4, 21, 4, 27, 4, 28, 4, 30, 2, 0, 9, 29, 9, 34, 9, 38, 0, 2, 1, 9, 1, 13, 5, 35]
Fitness value of the best solution = 42


In [34]:
new_result = solver.re_run_agent()
print("Parameters of the best solution : {solution}".format(solution=new_result[0]))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(new_result[1])))

Done
Parameters of the best solution : [9, 0, 9, 1, 9, 5, 9, 10, 6, 0, 6, 2, 6, 6, 6, 8, 8, 0, 8, 3, 6, 11, 9, 13, 3, 0, 3, 4, 6, 13, 6, 15, 1, 0, 3, 8, 3, 17, 3, 21, 6, 17, 6, 22, 3, 25, 3, 29, 0, 0, 0, 1, 8, 11, 6, 24, 2, 0, 8, 20, 8, 28, 8, 30, 5, 0, 9, 15, 6, 27, 6, 29, 5, 2, 6, 31, 3, 33, 3, 37]
Fitness value of the best solution = 41


In [15]:
result = new_result[0]
fitness = new_result[1]

In [35]:
schedule = GAToScheduleTranslator().translate(result, all_jobs, env, orders)
schedule.created_in = env
schedule.created_for = orders
visualize_schedule_demo(schedule, env, orders)
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

Fitness Values:
Makespan: 42
Tardiness: 0
Deviation: 709
Idle Time: 97
Profit: 5000


In [22]:
import pickle

#solver.environment = None # make file smaller, since the schedule already knows the environment and the orders
#solver.orders = None
#schedule.created_by = solver
schedule.created_by = 'HybridAgentSolver'
schedule.evaluation_results = []
schedule.evaluation_results.append(('makespan', fitness_values[0]))
schedule.evaluation_results.append(('tardiness', fitness_values[1]))
schedule.evaluation_results.append(('deviation', fitness_values[2]))
schedule.evaluation_results.append(('idle_time', fitness_values[3]))
schedule.evaluation_results.append(('profit', fitness_values[4]))
save_file = True
if(save_file):
    with open(f'results/HybridSolverTesting.pickle', 'wb') as outfile:
        pickled_data = pickle.dump(schedule, outfile)